# Python Functions: First-Class Citizens
## Overview

In Python, functions are **First-Class Citizens**. This means they are treated exactly like any other object (integer, string, list, etc.). They can be:

1. Assigned to variables.
2. Passed as arguments to other functions.
3. Returned from other functions.
4. Stored in data structures.

This behavior is distinct from languages like C or Java (pre-8), where functions are distinct language constructs separate from data. Understanding this is prerequisite to mastering **Higher-Order Functions**, **Decorators**, and **Callbacks**.

---

## 1. Functions are Objects

Since every function is an object, it possesses memory address identity, type, and attributes. We can inspect these attributes at runtime using introspection.

### Introspection

You can access metadata about a function using standard attribute access (often via "dunder" or magic attributes).

```python
def calculate_velocity(dist, time):
    """
    Calculates velocity given distance and time.
    Returns value in m/s.
    """
    return dist / time

# 1. Verify it is an instance of the 'function' class
print(type(calculate_velocity))
# Output: <class 'function'>

# 2. Access the Docstring (Documentation) programmatically
print(calculate_velocity.__doc__)
# Output: Calculates velocity given distance and time...

# 3. Access the function's name as a string
print(calculate_velocity.__name__)
# Output: 'calculate_velocity'

```

In [1]:
print(print.__name__)

print


In [3]:
print(print.__doc__)

Prints the values to a stream, or to sys.stdout by default.

  sep
    string inserted between values, default a space.
  end
    string appended after the last value, default a newline.
  file
    a file-like object (stream); defaults to the current sys.stdout.
  flush
    whether to forcibly flush the stream.


## 2. Function Aliasing (Assigning to Variables)

Because a function is just an object residing in memory, the function name (`def my_func`) is merely a variable pointing to that object. You can assign that object to a new variable, creating an **alias**.

### The Parentheses Distinction `()`

* **`func()`**: **Calls** the function (executes the code).
* **`func`**: **References** the function object (does not execute).

### Engineering Example: Dynamic Dispatch

This is commonly used to switch behaviors dynamically without complex `if/else` chains.

```python
# Standard built-in function
print("Standard Print")

# 1. Assign the 'print' object to a new variable named 'log'
log = print

# 2. Use 'log' exactly as you would use 'print'
log("This is a message via the alias.")
# Output: This is a message via the alias.

# 3. Verify they point to the same memory address
print(id(print) == id(log))
# Output: True

```

### Engineering Example: Input Wrapper

We can alias the `input` function to creating semantic names for user interaction.

```python
# Alias 'input' to 'prompt_user'
prompt_user = input

# The code reads more like a sentence
user_name = prompt_user("Please enter your system ID: ")
print(f"System ID {user_name} recorded.")

```

---

## 3. Custom Function Aliasing

This behavior applies equally to user-defined functions.

```python
def database_connect():
    return "Connected to DB"

def mock_connect():
    return "Connected to MOCK DB"

# Configuration flag
IS_TESTING = True

# Assign the function based on configuration
# Note: We are NOT calling the functions (no parentheses), just assigning objects
if IS_TESTING:
    get_connection = mock_connect
else:
    get_connection = database_connect

# Execute the aliased function
status = get_connection()
print(status)
# Output: Connected to MOCK DB

```

## Summary

* **Everything is an Object:** Functions are instances of the `function` class.
* **Reference vs. Call:** `my_func` is the object; `my_func()` invokes it.
* **Aliasing:** You can rename or pass functions around just like variables `x` or `y`.

In [7]:
log = print
log("starting the system")
log(id(log),id(print))

starting the system


In [9]:
log1 = print() # if we use paranthesis while assigning the function to a varaible, it is calling a function and return value is assigned to the variable.
# IN the above case print returns none
log1("hello")

TypeError: 'NoneType' object is not callable

# Function as a Parameter (Higher-Order Functions)

## Overview

In Python, because functions are **First-Class Citizens** (objects), they can be passed as arguments to other functions. This allows you to write **Higher-Order Functions**—functions that accept other functions to dynamically alter their behavior.

This is a powerful concept. It allows you to pass **logic** (behavior) into a function, rather than just **data**.

---

## 1. The Concept: Passing Behavior

Standard functions accept data (integers, strings) and process it. Higher-order functions accept a "strategy" or "instruction" on *how* to process that data.

### Syntax Logic

* **Passing:** `wrapper_function(my_func)`  Passes the function object.
* **Executing:** `wrapper_function(my_func())`  Passes the *result* of the function (not the function itself).

---

## 2. Engineering Example: A Dynamic Calculator

Instead of writing one massive function with many `if/elif` statements to handle addition, subtraction, and multiplication, we can create a generic `compute` engine that accepts *any* operation function.

```python
# 1. Define standard logic functions (The "Strategies")
def add(a, b):
    return a + b

def multiply(a, b):
    return a * b

def custom_power_logic(a, b):
    """A custom strategy defined later"""
    return (a ** b) + 10

# 2. Define the Higher-Order Function (The "Engine")
def compute(action_func, x, y):
    """
    Takes a function 'action_func' and applies it to x and y.
    """
    print(f"--> Executing strategy: {action_func.__name__}")
    return action_func(x, y)

# 3. Dynamic Execution
# Pass the 'add' function object
result_1 = compute(add, 10, 5)
print(f"Result: {result_1}")  # Output: 15

# Pass the 'multiply' function object
result_2 = compute(multiply, 10, 5)
print(f"Result: {result_2}")  # Output: 50

# Pass the custom logic
result_3 = compute(custom_power_logic, 2, 3)
print(f"Result: {result_3}")  # Output: 18 (2^3 + 10)

```

### Analysis

* The `compute` function is **decoupled** from the specific math logic. It doesn't care *what* happens to `x` and `y`, only that `action_func` can handle them.
* This implements the **Strategy Design Pattern**: We swap algorithms (add, multiply) at runtime without changing the code of the engine (`compute`).

---

## 3. Real-World Application: Callbacks

In software engineering, functions passed as parameters are often called **Callbacks**. They are heavily used in:

1. **Event Handling:** "When the user clicks this button, execute `save_file`."
2. **Asynchronous Operations:** "Download this file, and when finished, run `process_data`."
3. **Data Transformation:** Python's built-in `map()` and `filter()` functions are higher-order functions.

### Example: Sort with Custom Logic

The built-in `sorted()` function is a higher-order function. It accepts a `key` parameter, which is a function that tells `sorted` how to compare items.

```python
data = ["apple", "Banana", "cherry"]

# Pass the 'len' function as a parameter
# Logic: Sort by length of string, not alphabetical order
sorted_by_len = sorted(data, key=len)

print(sorted_by_len)
# Output: ['apple', 'cherry', 'Banana'] (sorted by length 5, 6, 6)

```

## Summary

| Term | Definition |
| --- | --- |
| **Higher-Order Function** | A function that accepts another function as an argument (e.g., `compute`). |
| **Callback** | The function being passed in (e.g., `add`). |
| **Benefit** | Increases flexibility and reusability by separating the "process flow" from the "specific logic." |

# Returning Functions (Function Factories)

## Overview

We have established that functions are **First-Class Citizens** (objects). This implies a powerful symmetry: just as we can pass a function *into* another function (as an argument), we can also **return a function** *out of* another function.

This concept is the foundation of **Function Factories** and **Closures**. It allows you to write functions that write *other* functions dynamically.

### Definition: Higher-Order Function

A function is classified as **Higher-Order** if it meets **at least one** of these criteria:

1. It accepts a function as a parameter (e.g., `map`, `filter`, callbacks).
2. It **returns a function** as its result.

---

## 1. The Mechanism: Returning References

When returning a function, we must return the **object reference**, not the result of calling it.

* **`return inner_func()`**: Executes the function and returns its *result* (e.g., `None`, `int`, etc.).
* **`return inner_func`**: Returns the **function object itself**, ready to be assigned and called later.

### Basic Syntax

```python
def function_factory():
    def generated_function():
        return "I am the inner function executing!"
    
    # Return the OBJECT, do not call it '()'
    return generated_function

# 1. Call the factory
# 'my_func' now holds the 'generated_function' object
my_new_func = function_factory()

# 2. Execute the returned function
print(my_new_func())
# Output: I am the inner function executing!

```

---

## 2. Engineering Use Case: The "Factory" Pattern

The most practical application of returning functions is creating **Function Factories**—functions that generate specialized behavior based on initial configuration.

### Example: A Multiplier Generator

Instead of writing separate functions for `double()`, `triple()`, and `quadruple()`, we write one `multiplier_maker` that generates them for us.

```python
def create_multiplier(factor):
    """
    Higher-Order Function: Returns a specialized function
    that multiplies numbers by the given 'factor'.
    """
    
    # Inner function 'remembers' the 'factor' variable
    # This is technically a Closure (more on this later)
    def multiplier(number):
        return number * factor
    
    return multiplier

# Generate specific functions
double = create_multiplier(2)  # Returns a function that multiplies by 2
triple = create_multiplier(3)  # Returns a function that multiplies by 3

# Use the generated functions
print(f"Double 10: {double(10)}") # Output: 20
print(f"Triple 10: {triple(10)}") # Output: 30

# Verify they are distinct objects
print(f"Double ID: {id(double)}")
print(f"Triple ID: {id(triple)}")
# Output: Different memory addresses

```

### Why is this useful?

1. **DRY Principle:** You write the logic once (`number * factor`) and reuse it infinitely.
2. **Configuration:** You can "configure" functions at runtime (e.g., creating a `logger` function that is pre-configured to write to a specific file).

---

## Summary of Functional Features

| Feature | Description | Code Snippet |
| --- | --- | --- |
| **First-Class Object** | Function is treated as a variable. | `x = my_func` |
| **Nested Function** | Function defined inside another. | `def outer(): def inner(): ...` |
| **Function as Parameter** | Passing logic *into* a function. | `compute(add, 10, 5)` |
| **Returning Function** | Generating logic *out of* a function. | `return inner_func` |

These concepts are the building blocks for **Decorators**, which is one of Python's most powerful features for modifying code behavior cleanly.

# Python Closures

## Overview

A **Closure** is a powerful functional programming concept where a nested function "remembers" and can access variables from its enclosing (outer) function, even after the outer function has finished executing.

In simpler terms: A closure is a function with a **backpack** of data attached to it. It allows functions to have "state" without using global variables or classes.

---

## 1. The Three Rules of Closures

For a function to be considered a closure, it must satisfy three specific conditions:

1. **Nested Function:** There must be an inner function defined inside an outer function.
2. **Access Enclosing Scope:** The inner function must reference variables defined in the outer function.
3. **Return the Function:** The outer function must return the **inner function** itself (not the result).

### Basic Anatomy

```python
def outer_function(message):
    # This variable belongs to the outer scope
    text = message
    
    def inner_function():
        # The inner function accesses 'text' from the outer scope
        print(f"Message from closure: {text}")
    
    # Return the FUNCTION OBJECT (no parentheses)
    return inner_function

# 1. Execution phase of outer function
# 'my_closure' now holds the inner function WITH the environment "Hello World"
my_closure = outer_function("Hello World")

# 2. Execution phase of inner function
# Even though outer_function has finished, 'my_closure' remembers "Hello World"
my_closure()
# Output: Message from closure: Hello World

```

---

## 2. Modifying Closure State: `nonlocal`

By default, an inner function can **read** variables from the outer scope. However, if you try to **modify** an immutable variable (like an integer or string) from the outer scope, Python treats it as a new local variable definition, breaking the closure.

To solve this, we use the `nonlocal` keyword. This tells Python: *"I am not defining a new variable; I want to modify the specific variable in the nearest enclosing scope."*

### Engineering Example: The Counter Factory

A classic use case for closures is creating isolated counters or state machines.

```python
def create_counter():
    """
    A Factory that creates independent counter functions.
    """
    count = 0  # This variable persists!

    def counter():
        nonlocal count  # Permission to modify the outer 'count'
        count += 1
        return count

    return counter

# Create two independent counters
# Each call to create_counter() creates a NEW scope with a NEW 'count' variable
counter_a = create_counter()
counter_b = create_counter()

print(f"Counter A: {counter_a()}") # 1
print(f"Counter A: {counter_a()}") # 2
print(f"Counter A: {counter_a()}") # 3

# Counter B is completely independent
print(f"Counter B: {counter_b()}") # 1

```

**Why is this cool?**
You have created data privacy. No other part of your code can mess with the `count` variable inside `counter_a`. It is completely hidden (encapsulated) inside the closure.

---

## 3. Deep Dive: How does it work?

When a function returns, its local variables are usually destroyed (garbage collected). However, if an inner function is holding onto a reference to those variables, Python detects this dependence.

It bundles the inner function code *plus* the references to the outer variables into a **Closure Cell**. You can actually inspect this introspection.

```python
# Inspecting the closure storage
print(counter_a.__closure__)
# Output: (<cell at 0x...: int object at 0x...>,)

# Seeing the value inside
print(counter_a.__closure__[0].cell_contents)
# Output: 3 (The current count)

```

---

## 4. Closures vs. Classes

Closures and Object-Oriented Programming (Classes) often solve the same problem: **binding data to functionality.**

| Approach | Use Case |
| --- | --- |
| **Closure** | Best when you have a **single method** or simple interface. It is lightweight and syntactically cleaner (less boilerplate). |
| **Class** | Best when you need multiple methods, inheritance, or complex state management. |

### Comparison

**The Class Way:**

```python
class Multiplier:
    def __init__(self, factor):
        self.factor = factor
        
    def execute(self, number):
        return number * self.factor

doubler = Multiplier(2)
print(doubler.execute(10))

```

**The Closure Way (Cleaner for simple tasks):**

```python
def make_multiplier(factor):
    def multiplier(number):
        return number * factor
    return multiplier

doubler = make_multiplier(2)
print(doubler(10))

```

## Summary

* **Closures** preserve the environment in which they were created.
* Use **`nonlocal`** if the inner function needs to modify outer variables.
* They are excellent for **data hiding**, **factories**, and replacing small classes.
* They are the fundamental mechanism behind **Decorators** (which we will cover next).

In [17]:
#simple closure function

def outer_function(message):

  def inner_function():
    print("From inner function",message )

  return inner_function


hello = outer_function("Hello World")
hello()

print(id(hello))

gm = outer_function("Good morning")

print(id(gm))
gm()

hello = outer_function("Hello World")

print(id(hello))
hello()


From inner function Hello World
133869955538336
133869955534656
From inner function Good morning
133869955538176
From inner function Hello World


In [19]:
def counter_creater():
  count =0

  def counter():
    nonlocal count
    count += 1
    return count

  return counter


apples = counter_creater()
oranges = counter_creater()

print(apples(),apples(), apples(), oranges())


1 2 3 1


# Python Decorators

## Overview

A **Decorator** is a powerful design pattern in Python that allows you to modify or extend the behavior of a function (or class) without permanently changing its source code.

Syntactically, a decorator is a **Closure** combined with **Higher-Order Functions**. It takes a function as an argument, creates a "wrapper" function that adds functionality (before and after the original function runs), and returns that wrapper.

---

## 1. The Anatomy of a Decorator

A decorator is essentially a "wrapper" logic. It intercepts the function call, executes some code, runs the original function, executes more code, and then returns the result.

### The Formula

1. **Input:** A function (`original_func`).
2. **Inner Logic:** A nested wrapper function that adds pre/post-processing.
3. **Output:** The wrapper function (which replaces the original).

### Code Structure (The Manual Way)

Before looking at the `@` syntax, it is crucial to understand what happens under the hood.

```python
def my_decorator(target_function):
    """
    A simple decorator that adds 'decoration'
    before and after the target function runs.
    """
    
    def wrapper():
        print("--- [Log] Start of Execution ---") # Pre-processing
        
        target_function()  # Call the original function
        
        print("--- [Log] End of Execution ---")   # Post-processing
    
    return wrapper  # Return the new function

# 1. Define a standard function
def say_hello():
    print("Hello, System!")

# 2. "Decorate" it manually
# We pass 'say_hello' into the factory, and get 'wrapper' back.
# We overwrite the name 'say_hello' with the new behavior.
say_hello = my_decorator(say_hello)

# 3. Call the modified function
say_hello()

```

**Output:**

```text
--- [Log] Start of Execution ---
Hello, System!
--- [Log] End of Execution ---

```

---

## 2. The Syntactic Sugar: `@decorator`

Python provides a shorthand syntax using the `@` symbol. This eliminates the need for the manual reassignment line (`func = dec(func)`).

When you place `@decorator_name` above a function definition, Python automatically passes that function into the decorator and replaces it with the result.

### Engineering Example: Execution Timer

A common use case for decorators is performance monitoring (benchmarking).

```python
import time

def timer_decorator(func):
    """
    Measures the execution time of the decorated function.
    """
    def wrapper():
        start_time = time.time()       # 1. Before Logic
        
        func()                         # 2. Original Logic
        
        end_time = time.time()         # 3. After Logic
        print(f"Execution took: {end_time - start_time:.5f} seconds")
    
    return wrapper

# Apply the decorator using the @ syntax
@timer_decorator
def heavy_computation():
    print("Running complex task...")
    time.sleep(0.5) # Simulate work

# Execution
heavy_computation()

```

**What actually happened?**
Calling `heavy_computation()` actually called `wrapper()`. The original logic was safely tucked inside.

---

## 3. Decorator vs. Closure

It is easy to confuse the two because their structure is nearly identical. The key difference lies in **intent** and **parameters**.

| Feature | Closure | Decorator |
| --- | --- | --- |
| **Primary Input** | Data (e.g., `msg="Hello"`, `count=0`). | A **Function** object. |
| **Primary Goal** | To retain state (data persistence). | To modify/extend behavior (logic wrapper). |
| **Usage** | Creating factories or data hiding. | Logging, Authentication, Timing, Caching. |

### Comparison Code

```python
# CLOSURE: Takes data ('Welcome'), returns a printer
def make_printer(msg):
    def inner():
        print(msg)
    return inner

# DECORATOR: Takes a function, returns a wrapper
def make_pretty(func):
    def inner():
        print("***")
        func()
        print("***")
    return inner

```

## Summary

* **Decorators are Wrappers:** They allow you to "wrap" functionality around an existing function.
* **Non-Intrusive:** You can add logging, timing, or security checks to a function without touching the function's internal code.
* **`@` Syntax:** Use `@my_decorator` as a clean shortcut for `my_func = my_decorator(my_func)`.

# Python Callable Classes (`__call__`)

## Overview

In Python, functions are objects. Conversely, **objects can act like functions**.

A **Callable Class** is a class whose instances can be "called" using the parentheses syntax `()` just like a standard function. This is achieved by implementing the special "dunder" (double underscore) method: `__call__`.

This pattern allows you to create objects that have the **behavior** of a function but maintain **state** like an object. It is the Object-Oriented equivalent of a **Closure**.

---

## 1. The Mechanics: `__call__`

When you write `my_object(arg)`, Python internally translates this to `my_object.__call__(arg)`. If the class does not implement `__call__`, raising this syntax results in a `TypeError`.

### Syntax

```python
class MyCallable:
    def __call__(self, x, y):
        return x + y

# Usage
obj = MyCallable()
result = obj(5, 10) # Effectively calls obj.__call__(5, 10)

```

---

## 2. Engineering Example: A State-Aware Lookup

The transcript describes a `Day` class that acts as a lookup table. The class initializes a dictionary (state) and uses the call method to retrieve values (behavior).

### The Implementation

```python
class DayLookup:
    """
    A class that functions as a day-of-week converter.
    """
    def __init__(self):
        # 1. State: The dictionary is initialized once and stored in the object.
        self._days = {
            0: "Sunday",
            1: "Monday",
            2: "Tuesday",
            3: "Wednesday",
            4: "Thursday",
            5: "Friday",
            6: "Saturday"
        }

    def __call__(self, day_index):
        # 2. Behavior: This method runs when we use instance()
        # Using .get() is safer than direct access [] to handle errors
        return self._days.get(day_index, "Invalid Day")

# --- Execution ---

# 1. Instantiate the class (State is prepared)
get_day_name = DayLookup()

# 2. Use the object as if it were a function
print(f"Day 3 is: {get_day_name(3)}")  # Output: Wednesday
print(f"Day 0 is: {get_day_name(0)}")  # Output: Sunday

```

---

## 3. Callable Objects vs. Closures

The transcript correctly identifies that Callable Classes are functionally similar to **Closures**.

| Feature | Closure Function | Callable Class |
| --- | --- | --- |
| **State Storage** | Variables in the **Outer Scope** (Enclosing Function). | Instance Variables (`self.data`) in `__init__`. |
| **Execution Logic** | Code inside the **Inner Function**. | Code inside the `__call__` method. |
| **Use Case** | Lightweight, simple state. | Complex state, multiple helper methods, inheritance. |

### Why use a Class instead of a Closure?

While closures are concise, Callable Classes are often preferred in large systems because:

1. **Extensibility:** You can add other methods (e.g., `reset_days()`, `add_day()`) to the class easily.
2. **Type Checking:** It is easier to check `isinstance(obj, DayLookup)`.
3. **State Inspection:** You can easily inspect `obj._days` during debugging, whereas inspecting closure cells is difficult.

---

## Summary

* **The Concept:** Any object can be made callable by adding `__call__`.
* **The Benefit:** It combines the data retention of an object with the clean syntax of a function.
* **Common Uses:** Decorators with arguments, caching strategies, and strategy patterns.